# AEBS products in VirES

This notebook demonstrates access to the AEBS products data by means of the vires client.

For each VirES data collection the avaiable variables are listed. In addition, mapping the the variables of the source AEBS products is provided [*link to spec TBD*].

In [1]:
from viresclient import SwarmRequest

server_url = 'https://staging.viresdisc.vires.services/ows'


if True:
    # setup older client versions without AEBS support
    SwarmRequest.COLLECTIONS.update({
        'AEJ_LPL': ['SW_OPER_AEJ{}LPL_2F'.format(x) for x in 'ABC'],
        'AEJ_LPL:Quality': ['SW_OPER_AEJ{}LPL_2F:Quality'.format(x) for x in 'ABC'],
        'AEJ_LPS': ['SW_OPER_AEJ{}LPS_2F'.format(x) for x in 'ABC'],
        'AEJ_LPS:Quality': ['SW_OPER_AEJ{}LPS_2F:Quality'.format(x) for x in 'ABC'],
        'AEJ_PBL': ['SW_OPER_AEJ{}PBL_2F'.format(x) for x in 'ABC'],
        'AEJ_PBS': ['SW_OPER_AEJ{}PBS_2F'.format(x) for x in 'ABC'],
        'AOB_FAC': ['SW_OPER_AOB{}FAC_2F'.format(x) for x in 'ABC'],
    })
    SwarmRequest.PRODUCT_VARIABLES.update({
        'AEJ_LPL': ['Latitude_QD', 'Longitude_QD', 'MLT_QD', 'J_NE', 'J_QD'],
        'AEJ_LPL:Quality': ['RMS_misfit', 'Confidence'],
        'AEJ_LPS': ['Latitude_QD', 'Longitude_QD', 'MLT_QD', 'J_CF_NE', 'J_DF_NE', 'J_CF_SemiQD', 'J_DF_SemiQD', 'J_R'],
        'AEJ_LPS:Quality': ['RMS_misfit', 'Confidence'],
        'AEJ_PBL': ['Latitude_QD', 'Longitude_QD', 'MLT_QD', 'J_QD', 'Flags', 'PointType'],
        'AEJ_PBS': ['Latitude_QD', 'Longitude_QD', 'MLT_QD', 'J_DF_SemiQD', 'Flags', 'PointType'],
        'AOB_FAC': ['Latitude_QD', 'Longitude_QD', 'MLT_QD', 'Boundary_Flag', 'Quality', 'Pair_Indicator'],
    })

## AEJxLPL product

Avaiables Variables:

| VirES Variable | Source Product Variable | Unit | Dimension | Description | 
|---|---|---|---|---|
| `Timestamp` | `t` | $-$ | scalar |UTC time of observation |
| `Latitude` | `Latitude` | `Latitude` | $\text{deg}$ | scalar | ITRF geocentric latitude |
| `Longitude` | `Longitude` | $\text{deg}$ | scalar | ITRF geocentric longitude |
| `Latitude_QD` | `Latitude_QD` | $\text{deg}$ | scalar | Quasi-dipole latitude |
| `Longitude_QD` | `Longitude_QD` | $\text{deg}$ | scalar | Quasi-dipole longitude |
| `MLT_QD` | `MLT` | $\text{hour}$ | scalar | Magnetic local time (QD) |
| `J_NE` | `J` | $\text{A}\ \text{km}^{-1}$ | vector [2] | North and East component of horizontal sheet current density vector in the geocentric frame |
| `J_QD` | `J_QD` | $\text{A}\ \text{km}^{-1}$ | scalar | East component of horizontal sheet current density in QD frame |

Note: If the standard VirES `QDLat`, `QDLat` and `MLT` auxialiary variables default to the `Latitude_QD`, `Longitude_QD`, `MLT_QD` stored in AEBS products if they cannot be calculated from the times and locations. (Locations in the AEBS products are incomplete due to the missing `Radius` and they cannot be used to calucate the magnetic coordinates.)


In [2]:
request = SwarmRequest(server_url)
request.set_collection('SW_OPER_AEJALPL_2F')
request.set_products(
    measurements=['Latitude_QD', 'Longitude_QD', 'MLT_QD', 'J_NE', 'J_QD'],
    auxiliaries=['QDLat', 'QDLon', 'MLT'],
)

data = request.get_between(
    start_time='2015-06-01T00:00:00Z',
    end_time='2015-06-02T00:00:00Z',
).as_xarray()

print(data)

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.247MB)


<xarray.Dataset>
Dimensions:       (J_NE_dim1: 2, Timestamp: 2412)
Coordinates:
  * Timestamp     (Timestamp) datetime64[ns] 2015-06-01T00:00:15.688632727 ... 2015-06-01T23:33:26.110890627
Dimensions without coordinates: J_NE_dim1
Data variables:
    Spacecraft    (Timestamp) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    QDLon         (Timestamp) float64 -27.39 -31.73 -35.82 ... -74.72 -75.39
    Latitude_QD   (Timestamp) float64 -77.66 -77.26 -76.8 ... -52.17 -51.2
    Longitude_QD  (Timestamp) float64 -27.39 -31.73 -35.82 ... -74.72 -75.39
    J_NE          (Timestamp, J_NE_dim1) float64 58.18 -7.87 ... 6.264 -17.93
    MLT           (Timestamp) float64 17.6 17.32 17.05 ... 14.05 14.01 13.97
    QDLat         (Timestamp) float64 -77.66 -77.26 -76.8 ... -52.17 -51.2
    Latitude      (Timestamp) float64 -79.2 -78.22 -77.24 ... -50.34 -49.34
    Longitude     (Timestamp) float64 178.7 179.9 -179.2 ... -165.0 -164.9
    J_QD          (Timestamp) float64 -58.71 -64.28 -72.47 ...

## AEJxLPL product quality

The AEJxLPL quality information is accessible as a separate dataset.

Avaiables Variables:

| VirES Variable | Source Product Variable | Unit | Dimension | Description | 
|---|---|---|---|---|
| `Timestamp` | `t_qual` | $-$ | scalar | UTC timestamp of the quality record |
| `RMS_misfit` | `RMS_misfit` | $\text{nT}$ | scalar | Root Mean Square error misfit between observations and model values for the oval crossing |
| `Confidence` | `Confidence` | $-$ | scalar | Goodness of fit computed as 1 - (residual RMS / signal RMS) for the oval crossing |


In [3]:
request = SwarmRequest(server_url)
request.set_collection('SW_OPER_AEJALPL_2F:Quality')
request.set_products(
    measurements=['RMS_misfit', 'Confidence'],
)

data = request.get_between(
    start_time='2015-06-01T00:00:00Z',
    end_time='2015-06-02T00:00:00Z'
).as_xarray()

print(data)

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.037MB)

<xarray.Dataset>
Dimensions:     (Timestamp: 61)
Coordinates:
  * Timestamp   (Timestamp) datetime64[ns] 2015-06-01T00:05:37 ... 2015-06-01T23:30:25
Data variables:
    Spacecraft  (Timestamp) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    RMS_misfit  (Timestamp) float64 0.3433 0.5879 0.5471 ... 0.04362 0.07308
    Confidence  (Timestamp) float64 0.9642 0.9695 0.9793 ... 0.9834 0.9809
Attributes:
    Sources:         ['SW_OPER_AEJALPL_2F_20150531T000000_20150531T235959_010...
    MagneticModels:  []
    RangeFilters:    []


## AEJxLPS product

Avaiables Variables:

| VirES Variable | Source Product Variable | Unit | Dimension | Description | 
|---|---|---|---|---|
| `Timestamp` | `t` | $-$ | scalar |UTC time of observation |
| `Latitude` | `Latitude` | `Latitude` | $\text{deg}$ | scalar | ITRF geocentric latitude |
| `Longitude` | `Longitude` | $\text{deg}$ | scalar | ITRF geocentric longitude |
| `Latitude_QD` | `Latitude_QD` | $\text{deg}$ | scalar | Quasi-dipole latitude |
| `Longitude_QD` | `Longitude_QD` | $\text{deg}$ | scalar | Quasi-dipole longitude |
| `MLT_QD` | `MLT` | $\text{hour}$ | scalar | Magnetic local time (QD) |
| `J_CF_NE` | `J_CF` | $\text{A}\ \text{km}^{-1}$ | vector [2] | North and East component of the curl-free horizontal sheet current density vector in geocentric frame |
| `J_DF_NE` | `J_DF` | $\text{A}\ \text{km}^{-1}$ | vector [2] | North and East component of the divergence-free horizontal sheet current density vector in geocentric frame |
| `J_CF_SemiQD` | `J_CF_SemiQD` | $\text{A}\ \text{km}^{-1}$ | scalar | North component of the curl-free horizontal sheet current density in semi-QD frame |
| `J_DF_SemiQD` | `J_DF_SemiQD` | $\text{A}\ \text{km}^{-1}$ | scalar | East component of the divergence-free horizontal sheet current density in semi-QD frame |
| `J_R` | `J_r` | $\text{A}\ \text{km}^{-2}$ | scalar | Radial ionospheric current density at 110 km altitude in SemiQD frame |


Note: If the standard VirES `QDLat`, `QDLat` and `MLT` auxialiary variables default to the `Latitude_QD`, `Longitude_QD`, `MLT_QD` stored in AEBS products if they cannot be calculated from the times and locations. (Locations in the AEBS products are incomplete due to the missing `Radius` and they cannot be used to calucate the magnetic coordinates.)

In [4]:
request = SwarmRequest(server_url)
request.set_collection('SW_OPER_AEJALPS_2F')
request.set_products(
    measurements=[
        'Latitude_QD', 'Longitude_QD', 'MLT_QD',
        'J_CF_NE', 'J_DF_NE', 'J_CF_SemiQD', 'J_DF_SemiQD', 'J_R',
    ],
    auxiliaries=['QDLat', 'QDLon', 'MLT'],
)

data = request.get_between(
    start_time='2015-06-01T00:00:00Z',
    end_time='2015-06-02T00:00:00Z'
).as_xarray()

print(data)

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (4.836MB)


<xarray.Dataset>
Dimensions:       (J_CF_NE_dim1: 2, J_DF_NE_dim1: 2, Timestamp: 37416)
Coordinates:
  * Timestamp     (Timestamp) datetime64[ns] 2015-06-01 ... 2015-06-01T23:59:59
Dimensions without coordinates: J_CF_NE_dim1, J_DF_NE_dim1
Data variables:
    Spacecraft    (Timestamp) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    J_DF_SemiQD   (Timestamp) float64 -132.5 -131.9 -131.2 ... 25.7 25.81 25.84
    QDLon         (Timestamp) float64 -22.79 -23.08 -23.38 ... -102.3 -102.3
    J_DF_NE       (Timestamp, J_DF_NE_dim1) float64 132.5 1.824 ... -5.264 25.3
    J_R           (Timestamp) float64 -0.08894 -0.08894 ... -0.04155 -0.01852
    Latitude_QD   (Timestamp) float64 -78.32 -78.3 -78.28 ... 50.96 51.02 51.09
    Longitude_QD  (Timestamp) float64 -22.79 -23.08 -23.38 ... -102.3 -102.3
    MLT           (Timestamp) float64 17.9 17.88 17.86 17.84 ... 12.6 12.6 12.6
    J_CF_SemiQD   (Timestamp) float64 -22.12 -22.27 -22.42 ... 8.053 7.782 7.614
    QDLat         (Timestamp) 

## AEJxLPS product quality

The AEJxLPS quality information is accessible as a separate dataset.

Avaiables Variables:

| VirES Variable | Source Product Variable | Unit | Dimension | Description |
|---|---|---|---|---|
| `Timestamp` | `t_qual` | $-$ | scalar | UTC timestamp of the quality record |
| `RMS_misfit` | `RMS_misfit` | $\text{nT}$ | scalar | Root Mean Square error misfit between observations and model values for the oval crossing |
| `Confidence` | `Confidence` | $-$ | scalar | Goodness of fit computed as 1 - (residual RMS / signal RMS) for the oval crossing |


In [5]:
request = SwarmRequest(server_url)
request.set_collection('SW_OPER_AEJALPS_2F:Quality')
request.set_products(
    measurements=['RMS_misfit', 'Confidence'],
)

data = request.get_between(
    start_time='2015-06-01T00:00:00Z',
    end_time='2015-06-02T00:00:00Z'
).as_xarray()

print(data)

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.037MB)

<xarray.Dataset>
Dimensions:     (Timestamp: 61)
Coordinates:
  * Timestamp   (Timestamp) datetime64[ns] 2015-06-01T00:34:39 ... 2015-06-01T23:59:42
Data variables:
    Spacecraft  (Timestamp) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    RMS_misfit  (Timestamp) float64 2.298 4.103 2.858 3.35 ... 0.4935 1.32 2.046
    Confidence  (Timestamp) float64 0.8908 0.911 0.7526 ... 0.7863 0.3651 0.9071
Attributes:
    Sources:         ['SW_OPER_AEJALPS_2F_20150531T000000_20150531T235959_010...
    MagneticModels:  []
    RangeFilters:    []


## AEJxPBL product

| VirES Variable | Source Product Variable | Unit | Dimension | Description | 
|---|---|---|---|---|
| `Timestamp` | `t_Peak`, `t_EB`, `t_PB` | $-$ | scalar |UTC time of observation |
| `Latitude` | `Latitude_Peak`, `Latitude_EB`, `Latitude_PB` | $\text{deg}$ | scalar | ITRF geocentric latitude |
| `Longitude` | `Longitude_Peak`, `Longitude_EB`, `Longitude_PB` | $\text{deg}$ | scalar | ITRF geocentric longitude |
| `Latitude_QD` | `Latitude_Peak_QD`, `Latitude_EB_QD`, `Latitude_PB_QD` | $\text{deg}$ | scalar | Quasi-dipole latitude |
| `Longitude_QD` | `Longitude_Peak_QD`, `Longitude_EB_QD`, `Longitude_PB_QD` | $\text{deg}$ | scalar | Quasi-dipole longitude |
| `MLT_QD` | `MLT_Peak`, `MLT_EB`, `MLT_PB` | $\text{hour}$ | scalar | Magnetic local time (QD) |
| `J_QD` | `J` | $\text{A}\ \text{km}^{-1}$ | scalar | Peak eastward sheet current intensity in QD frame. Set to `NaN` for equatorial and polar boundaries. |
| `Flags` | `Flags` | $-$ | scalar | Quality indicator (for more details of the binary flags see the product specification) |
| `PointType` | n/a | $-$ | scalar | Point type (see the details below) |

`Latitude`, `Longitude`, `Latitude_QD`, `Longitude_QD`, `MLT_QD`, and `J_QD` are set to `NaN` if the EJ boubdaries have not been detected (also see the `Flags` specification).

The multi-timestamp table structure of the source AEJxPBL products is transormed into a simple time-series. The newly introduced `PointType` variable determines the type of the samples. The meaning of the bit flags is as follows:


| Bit No. | False (0) | True (1) | Note
|---|---|---|---|
| bit 0 (1) | WEJ | EEJ |
| bit 1 (2) | Peak | Boundary |
| bit 2 (4) | Equatorial boundary | Polar boundary | only if bit 1 set
| bit 3 (8) | Boundary pair start | Boundary pair end | only if bit 1 set

These are the possible `PointType` values:

| Value  | Hex. | Bin. | Meaning |
|---|---|---|---|
| 0 | 0x0 | 0b0000 | WEJ peak (minimum) |
| 1 | 0x1 | 0b0001 | EEJ peak (maximum) |
| 2 | 0x2 | 0b0010 | WEJ equatorial boundary (pair start) |
| 3 | 0x3 | 0b0011 | EEJ equatorial boundary (pair start) |
| 6 | 0x6 | 0b0110 | WEJ polar boundary (pair start) |
| 7 | 0x7 | 0b0111 | EEJ polar boundary (pair start) |
| 10 | 0xA | 0b1010 | WEJ equatorial boundary (pair end) |
| 11 | 0xB | 0b1011 | EEJ equatorial boundary (pair end) |
| 14 | 0xE | 0b1110 | WEJ polar boundary (pair end) |
| 15 | 0xF | 0b1111 | EEJ polar boundary (pair end) |

Note: If the standard VirES `QDLat`, `QDLat` and `MLT` auxialiary variables default to the `Latitude_QD`, `Longitude_QD`, `MLT_QD` stored in AEBS products if they cannot be calculated from the times and locations. (Locations in the AEBS products are incomplete due to the missing `Radius` and they cannot be used to calucate the magnetic coordinates.)

In [6]:
request = SwarmRequest(server_url)
request.set_collection('SW_OPER_AEJAPBL_2F')
request.set_products(
    measurements=[
        'Latitude_QD', 'Longitude_QD', 'MLT_QD',
         'J_QD', 'Flags', 'PointType',
    ],
    auxiliaries=['QDLat', 'QDLon', 'MLT'],
)

data = request.get_between(
    start_time='2015-06-01T00:00:00Z',
    end_time='2015-06-02T00:00:00Z'
).as_xarray()

print(data)

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.115MB)


<xarray.Dataset>
Dimensions:       (Timestamp: 366)
Coordinates:
  * Timestamp     (Timestamp) datetime64[ns] 2015-06-01T00:02:07.298531294 ... 2015-06-01T23:33:41.800961018
Data variables:
    Spacecraft    (Timestamp) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    Flags         (Timestamp) uint8 80 80 80 80 80 80 ... 113 113 113 113 113
    PointType     (Timestamp) uint16 7 1 11 6 0 10 2 0 ... 14 15 6 3 1 0 15 10
    QDLon         (Timestamp) float64 -52.56 -59.43 -64.89 ... -74.03 nan -76.03
    Latitude_QD   (Timestamp) float64 -73.62 -71.39 -68.96 ... -53.14 nan -50.22
    Longitude_QD  (Timestamp) float64 -52.56 -59.43 -64.89 ... -74.03 nan -76.03
    MLT           (Timestamp) float64 15.95 15.51 15.15 ... 14.05 nan 13.93
    QDLat         (Timestamp) float64 -73.62 -71.39 -68.96 ... -53.14 nan -50.22
    Latitude      (Timestamp) float64 -72.2 -69.33 -66.45 ... -51.34 nan -48.34
    Longitude     (Timestamp) float64 -176.0 -174.9 -174.2 ... -165.0 nan -164.9
    J_QD   

## AEJxPBS product

Avaiables Variables:

| VirES Variable | Source Product Variable | Unit | Dimension | Description | 
|---|---|---|---|---|
| `Timestamp` | `t_Peak`, `t_EB`, `t_PB` | $-$ | scalar |UTC time of observation |
| `Latitude` | `Latitude_Peak`, `Latitude_EB`, `Latitude_PB` | $\text{deg}$ | scalar | ITRF geocentric latitude |
| `Longitude` | `Longitude_Peak`, `Longitude_EB`, `Longitude_PB` | $\text{deg}$ | scalar | ITRF geocentric longitude |
| `Latitude_QD` | `Latitude_Peak_QD`, `Latitude_EB_QD`, `Latitude_PB_QD` | $\text{deg}$ | scalar | Quasi-dipole latitude |
| `Longitude_QD` | `Longitude_Peak_QD`, `Longitude_EB_QD`, `Longitude_PB_QD` | $\text{deg}$ | scalar | Quasi-dipole longitude |
| `MLT_QD` | `MLT_Peak`, `MLT_EB`, `MLT_PB` | $\text{hour}$ | scalar | Magnetic local time (QD) |
| `J_DF_SemiQD` | `J_DF` | $\text{A}\ \text{km}^{-1}$ | scalar | Peak eastward sheet current intensity in semi-QD frame.  Set to `NaN` for equatorial and polar boundaries. |
| `Flags` | `Flags` | $-$ | scalar | Quality indicator (for more details of the binary flags see the product specification) |
| `PointType` | n/a | $-$ | scalar | Point type (see the details below) |

`Latitude`, `Longitude`, `Latitude_QD`, `Longitude_QD`, `MLT_QD`, and `J_DF_SemiQD` are set to `NaN` if the EJ boubdaries have not been detected (also see the `Flags` specification).

The multi-timestamp table structure of the source AEJxPBS products is transormed into a simple time-series. The newly introduced `PointType` variable determines the type of the samples. The meaning of the bit flags is as follows:

| Bit No. | False (0) | True (1) | Note
|---|---|---|---|
| bit 0 (1) | WEJ | EEJ |
| bit 1 (2) | Peak | Boundary |
| bit 2 (4) | Equatorial boundary | Polar boundary | only if bit 1 set
| bit 3 (8) | Boundary pair start | Boundary pair end | only if bit 1 set

These are the possible `PointType` values:

| Value  | Hex. | Bin. | Meaning |
|---|---|---|---|
| 0 | 0x0 | 0b0000 | WEJ peak (minimum) |
| 1 | 0x1 | 0b0001 | EEJ peak (maximum) |
| 2 | 0x2 | 0b0010 | WEJ equatorial boundary (pair start) |
| 3 | 0x3 | 0b0011 | EEJ equatorial boundary (pair start) |
| 6 | 0x6 | 0b0110 | WEJ polar boundary (pair start) |
| 7 | 0x7 | 0b0111 | EEJ polar boundary (pair start) |
| 10 | 0xA | 0b1010 | WEJ equatorial boundary (pair end) |
| 11 | 0xB | 0b1011 | EEJ equatorial boundary (pair end) |
| 14 | 0xE | 0b1110 | WEJ polar boundary (pair end) |
| 15 | 0xF | 0b1111 | EEJ polar boundary (pair end) |

Note: If the standard VirES `QDLat`, `QDLat` and `MLT` auxialiary variables default to the `Latitude_QD`, `Longitude_QD`, `MLT_QD` stored in AEBS products if they cannot be calculated from the times and locations. (Locations in the AEBS products are incomplete due to the missing `Radius` and they cannot be used to calucate the magnetic coordinates.)

In [7]:
equest = SwarmRequest(server_url)
request.set_collection('SW_OPER_AEJAPBS_2F')
request.set_products(
    measurements=[
        'Latitude_QD', 'Longitude_QD', 'MLT_QD',
         'J_DF_SemiQD', 'Flags', 'PointType',
    ],
    auxiliaries=['QDLat', 'QDLon', 'MLT'],
)

data = request.get_between(
    start_time='2015-06-01T00:00:00Z',
    end_time='2015-06-02T00:00:00Z'
).as_xarray()

print(data)

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.115MB)


<xarray.Dataset>
Dimensions:       (Timestamp: 364)
Coordinates:
  * Timestamp     (Timestamp) datetime64[ns] 2015-06-01T00:01:32 ... 2015-06-01T23:34:07
Data variables:
    Spacecraft    (Timestamp) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    J_DF_SemiQD   (Timestamp) float64 nan nan 96.41 nan ... nan nan -7.49 nan
    Flags         (Timestamp) uint8 96 96 96 96 32 32 32 ... 72 72 72 72 72 72
    PointType     (Timestamp) uint16 10 7 1 11 3 1 15 2 0 ... 0 14 7 1 11 6 0 10
    QDLon         (Timestamp) float64 -45.97 -45.97 -58.46 ... -73.9 -76.97
    Latitude_QD   (Timestamp) float64 -75.57 -75.57 -72.24 ... -54.38 -49.98
    Longitude_QD  (Timestamp) float64 -45.97 -45.97 -58.46 ... -73.9 -76.97
    MLT           (Timestamp) float64 16.38 16.38 15.57 ... 14.37 14.05 13.87
    QDLat         (Timestamp) float64 -75.57 -75.57 -72.24 ... -54.38 -49.98
    Latitude      (Timestamp) float64 -74.35 -74.35 -69.9 ... -52.45 -47.95
    Longitude     (Timestamp) float64 -179.0 -179.0

## AEJxPBS product - Ground magnetic disturbance

The AEJxPBS ground magnetic disturbance is avaialble as a separate dataset.

Avaiables Variables:


| VirES Variable | Source Product Variable | Unit | Dimension | Description |
|---|---|---|---|---|
| `Timestamp` | n/a | $-$ | scalar | UTC timestamp ⃰ | 
| `Latitude` | `Latitude_B` | $\text{deg}$ | scalar | **Geodetic** latitude ⃰ ⃰ |
| `Longitude` | `Longitude_B` | $\text{deg}$ | scalar | **Geodetic** longitude ⃰ ⃰ |
| `B_NE` | `B` | $\text{nT}$ | vector [2] | Peak value of the horizontal ground magnetic disturbance in the **geodetic** NE frame ⃰ ⃰ |

 ⃰ The ground magnetic disturbance timestamp is not present in the original product. It is assigned from the corresponding peak and therefore it may not excatly express the time of the disturbance event.

 ⃰ ⃰ Due to the missing elevation the geodetic coordinates cannot be reliably converted to ITRF geocentric ones.

In [8]:
SwarmRequest.COLLECTIONS['AEJ_PBS:GroundMagneticDisturbance'] = [
    'SW_OPER_AEJ{}PBS_2F:GroundMagneticDisturbance'.format(x) for x in 'ABC'
]

SwarmRequest.PRODUCT_VARIABLES['AEJ_PBS:GroundMagneticDisturbance'] = ['B']

request = SwarmRequest(server_url)
request.set_collection('SW_OPER_AEJAPBS_2F:GroundMagneticDisturbance')
request.set_products(
    measurements=['B'],
)

data = request.get_between(
    start_time='2015-06-01T00:00:00Z',
    end_time='2015-06-02T00:00:00Z'
).as_xarray()

print(data)

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.037MB)


<xarray.Dataset>
Dimensions:     (Timestamp: 121)
Coordinates:
  * Timestamp   (Timestamp) datetime64[ns] 2015-06-01T00:02:46 ... 2015-06-01T23:32:53
Data variables:
    Spacecraft  (Timestamp) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    Longitude   (Timestamp) float64 165.7 -144.9 -163.9 ... 34.37 -166.3 178.3
    Latitude    (Timestamp) float64 -82.66 85.2 79.07 ... -72.74 -52.89 -80.83
Attributes:
    Sources:         ['SW_OPER_AEJAPBS_2F_20150101T000000_20151231T235959_0101']
    MagneticModels:  []
    RangeFilters:    []


## AOBxFAC product

Avaiables Variables:

| Variable | Unit | Dimension | Description | 
|---|---|---|---|
| `Timestamp` | $-$ | scalar |UTC time of observation |
| `Latitude` | $\text{deg}$ | scalar | ITRF geocentric latitude |
| `Longitude` | $\text{deg}$ | scalar | ITRF geocentric longitude |
| `Latitude_QD` | $\text{deg}$ | scalar | Quasi-dipole latitude |
| `Longitude_QD` | $\text{deg}$ | scalar | Quasi-dipole longitude |
| `MLT_QD` | $\text{hour}$ | scalar | Magnetic local time (QD) |
| `Boundary_Flag` | $-$ | scalar | Indicator for equatorward (1) or poleward (2) boundary |
| `Pair_Indicator` | $-$ | scalar | Quality indicator (for more details of the binary flags see the product specification) |
| `Quality` | $-$ | vector [2] | Quality indicator of equatorward/poleward boundary (Pa, Sigma) |

Note: If the standard VirES `QDLat`, `QDLat` and `MLT` auxialiary variables default to the `Latitude_QD`, `Longitude_QD`, `MLT_QD` stored in AEBS products if they cannot be calculated from the times and locations. (Locations in the AEBS products are incomplete due to the missing `Radius` and they cannot be used to calucate the magnetic coordinates.)

In [9]:
request = SwarmRequest(server_url)
request.set_collection('SW_OPER_AOBAFAC_2F')
request.set_products(  
    measurements=[
        'Latitude_QD', 'Longitude_QD', 'MLT_QD',
        'Quality', 'Pair_Indicator',
    ],
    auxiliaries=['QDLat', 'QDLon', 'MLT'],
)

data = request.get_between(
    start_time='2015-06-01T00:00:00Z',
    end_time='2015-06-02T00:00:00Z'
).as_xarray()

print(data)

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.115MB)


<xarray.Dataset>
Dimensions:         (Quality_dim1: 2, Timestamp: 102)
Coordinates:
  * Timestamp       (Timestamp) datetime64[ns] 2015-06-01T00:02:10.500000 ... 2015-06-01T23:26:29.500000
Dimensions without coordinates: Quality_dim1
Data variables:
    Spacecraft      (Timestamp) object 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    Pair_Indicator  (Timestamp) int8 1 -1 1 -1 1 -1 1 -1 ... -1 0 1 -1 1 -1 0 0
    QDLon           (Timestamp) float64 -53.06 -59.62 -115.8 ... 23.12 -38.02
    Latitude_QD     (Timestamp) float64 -73.47 -71.3 70.39 ... -77.08 -74.39
    Longitude_QD    (Timestamp) float64 -53.11 -59.66 -115.8 ... 23.11 -38.06
    MLT             (Timestamp) float64 15.92 15.49 12.34 ... 1.213 20.36 16.35
    Quality         (Timestamp, Quality_dim1) float64 4.012 0.1697 ... 0.2392
    QDLat           (Timestamp) float64 -73.46 -71.29 70.4 ... -77.08 -74.39
    Latitude        (Timestamp) float64 -72.0 -69.22 72.77 ... -87.26 -75.78
    Longitude       (Timestamp) float64 -175.9